In [0]:
class Problem:
    #The abstract class for a formal problem.

    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        raise NotImplementedError

    #Transition model
    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return any(x is state for x in self.goal)
        else:
            return state == self.goal

    #Returns the cost of a soultion path from state1 to state2 via action
    #assuming cost c to get up to state1.
    #If the path doesn't matter, this func will only look at state2.
    #The defaul method costs 1 for every step in the path.
    def path_cost(self, c, state1, action, state2):
        return c + 1

class GraphProblem(Problem): 
    #The problem of searching a graph
    def __init__(self, initial, goal, graph):
        super().__init__(initial, goal) #initialize initial and goal states
        self.graph = graph

    def actions(self, A):
        #The actions are neighbors of a node A
        return list(self.graph.get(A).keys())

    def result(self, state, action): #Transition model
        #The result of going to a neighbor is just that neighbor
        return action

    def path_cost(self, cost_so_far, A, action, B):
        return cost_so_far + (self.graph.get(A, B) or np.inf)

class Graph:
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        #Make an undirected graph by adding symmetric edges to a directed one.
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, A, B, distance=1):
        #connect A to B with distance (A->B)
        #if undirectional, connect B to A with distance as well (B-A)
        self.connect1(A, B, distance)
        if not self.directed:
            self.connect1(B, A, distance)

    def connect1(self, A, B, distance):
        #connect A to B with distance (directional; one way)
        self.graph_dict.setdefault(A, {})[B] = distance

    def get(self, a, b=None):
        #get(a,b): returns the distance or None;
        #get(a): returns a dict of {node: distance} entries, possibly {}
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        #Create a search tree from a parent by an action
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "'{}'".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        #List the nodes reachable in one step from this node.
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        #Return the sequence of actions from the root to this node.
        return [node.action for node in self.path()[1:]]

    def path(self):
        #Return a list of nodes forming the path from the root to this node.
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [3]:
from collections import deque

def breadth_first_graph_search1(problem):
    node = Node(problem.initial); 
    fs = [node]

    frontier = deque([node]); 
    print("Frontier: ", frontier, "\n----------------")
    explored = []

    while frontier:
        node = frontier.pop(); 
        print("Frontier popped as: ", frontier, "-> Currently Exploring: ", node)
                
        explored.append(node.state); 
        print("Explored: ", explored) 
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state): 
                    fs.extend(child.solution())       
                    return(fs)
                frontier.append(child); 
                print("Frontier: ", frontier)
        print("----------------")                          
    
    return None
def breadth_first_graph_search2(problem):

    node = Node(problem.initial); 
    fs = [node]

    frontier = deque([node]); 
    print("Frontier: ", frontier, "\n----------------")
    explored = []

    while frontier:
        node = frontier.popleft(); 
        print("Frontier popped as: ", frontier, "-> Currently Exploring: ", node)

        explored.append(node.state); 
        print("Explored: ", explored) 

        if problem.goal_test(node.state):
            fs.extend(node.solution())             
            return(fs)
       
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child); 
                print("Frontier: ", frontier)
        print("----------------")                          
    
    return None
# 호출부분 ------------
romania_linkage = dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142))

#Create an undirected graph
romania_map = Graph(graph_dict=romania_linkage, directed=False) 
#Formulate graph problem (initial, goal, graph)
romania_problem = GraphProblem('Arad', 'Bucharest', romania_map)
#Problem solving by searching (BFS: time complexity = O(b^d))
FS = breadth_first_graph_search1(problem=romania_problem)

#Optimality
print("========================")
print("Final Solution:", FS)
print("========================")

Frontier:  deque(['Arad']) 
----------------
Frontier popped as:  deque([]) -> Currently Exploring:  'Arad'
Explored:  ['Arad']
Frontier:  deque(['Zerind'])
Frontier:  deque(['Zerind', 'Sibiu'])
Frontier:  deque(['Zerind', 'Sibiu', 'Timisoara'])
----------------
Frontier popped as:  deque(['Zerind', 'Sibiu']) -> Currently Exploring:  'Timisoara'
Explored:  ['Arad', 'Timisoara']
Frontier:  deque(['Zerind', 'Sibiu', 'Lugoj'])
----------------
Frontier popped as:  deque(['Zerind', 'Sibiu']) -> Currently Exploring:  'Lugoj'
Explored:  ['Arad', 'Timisoara', 'Lugoj']
Frontier:  deque(['Zerind', 'Sibiu', 'Mehadia'])
----------------
Frontier popped as:  deque(['Zerind', 'Sibiu']) -> Currently Exploring:  'Mehadia'
Explored:  ['Arad', 'Timisoara', 'Lugoj', 'Mehadia']
Frontier:  deque(['Zerind', 'Sibiu', 'Drobeta'])
----------------
Frontier popped as:  deque(['Zerind', 'Sibiu']) -> Currently Exploring:  'Drobeta'
Explored:  ['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta']
Frontier:  deque(